## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
import squidpy as sq
import qnorm
import os
import time as t
import itertools
import pickle
import anndata as ad
import warnings
import random as rand

from math import log
from anndata import AnnData
from scipy.stats import ttest_ind, loguniform, expon
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

D:\Anaconda\envs\bioinf_env\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 13.0.0. Please consider upgrading.
  warnings.warn(
D:\Anaconda\envs\bioinf_env\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


## Loading Data

In [2]:
with open('Model/model_genes.pkl', 'rb') as f:
    sc_genes = pickle.load(f)
            
with open('Model/model_yng.pkl', 'rb') as f:
    yng = pd.DataFrame(
        pickle.load(f).todense(),
        index = [0] * 400,
        columns = sc_genes
    )
    
with open('Model/model_old.pkl', 'rb') as f:
    old = pd.DataFrame(
        pickle.load(f).todense(),
        index = [1] * 400,
        columns = sc_genes
    )

yng = ad.AnnData(yng)
old = ad.AnnData(old)

sc.pp.normalize_total(yng, exclude_highly_expressed = True)
sc.pp.normalize_total(old, exclude_highly_expressed = True)
sc.pp.log1p(yng)
sc.pp.log1p(old)

yng = yng.to_df().astype(float)
old = old.to_df().astype(float)

D:\Anaconda\envs\bioinf_env\Lib\site-packages\anndata\_core\aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
D:\Anaconda\envs\bioinf_env\Lib\site-packages\anndata\_core\anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
D:\Anaconda\envs\bioinf_env\Lib\site-packages\anndata\_core\aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
D:\Anaconda\envs\bioinf_env\Lib\site-packages\anndata\_core\anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [3]:
%%capture

st1_slide_name = 'WSSKNKCLsp10446618' # Median age facial sample	
st2_slide_name = 'WSSKNKCLsp12887265' # Median age non-facial sample

slide_names = [st1_slide_name, st2_slide_name]
adatas = []
st_dfs = [None] * 2
st_obs = [None] * 2

for i, name in enumerate(slide_names):
    # Load and pre-process as per ganier
    adata = sq.read.visium(f'GanierDatasets/{name}', counts_file = 'filtered_feature_bc_matrix.h5')
    adata.var_names_make_unique()
    adata.var['mt'] = adata.var_names.str.startswith("MT-")
    sc.pp.calculate_qc_metrics(
        adata, qc_vars = ['mt'], inplace = True, log1p = False
    )
    sc.pp.filter_cells(adata, min_genes = 200)
    adata = adata[adata.obs.pct_counts_mt < 30, :].copy()

    adata = adata[:, adata.var.index.intersection(sc_genes)]
    
    # Store potentially useful data
    st_obs[i] = adata.obs
    sc.pp.normalize_total(adata, exclude_highly_expressed = True)
    sc.pp.log1p(adata)
    st_dfs[i] = pd.DataFrame(
                    np.asarray(adata.X.todense()),
                    columns = adata.var.index,
                    index = adata.obs.index
    )
    sq.gr.spatial_neighbors(adata)
    adatas.append(adata)

## Dataset Integration / Normalization

In [4]:
common_genes = st_dfs[0].columns.intersection(sc_genes)
yng = yng[common_genes]
old = old[common_genes]

trn_X = []
trn_y = []
tst_X = []
tst_y = []

raw_st_dfs = []

for i, df in enumerate(st_dfs):
    all_samples = pd.concat([yng, old, df], axis = 0)
    qnormed = qnorm.quantile_normalize(all_samples, axis = 0)

    yng_q = qnormed[ : yng.shape[0]]
    old_q = qnormed[yng.shape[0] : yng.shape[0] + old.shape[0]]

    yng_trn, yng_tst = train_test_split(yng_q, test_size = 0.1, random_state = 42)
    old_trn, old_tst = train_test_split(old_q, test_size = 0.1, random_state = 42)

    trn_X.append(pd.concat([yng_trn, old_trn]))
    trn_y.append(trn_X[i].index)
    tst_X.append(pd.concat([yng_tst, old_tst]))
    tst_y.append(tst_X[i].index)

    st_dfs[i] = qnormed[yng.shape[0] + old.shape[0] : ]

In [5]:
# Get scaled datasets
scaler = StandardScaler()
scale = lambda x : pd.DataFrame(scaler.fit_transform(x), index = x.index, columns = x.columns)

scld_trn_X = [scale(df) for df in trn_X]
scld_tst_X = [scale(df) for df in tst_X]
scld_st_dfs = [scale(df) for df in st_dfs]

## Marker Sets Loading

In [6]:
with open('Genesets/senmayo.pkl', 'rb') as f:
    senmayo = pickle.load(f)

with open('Genesets/fridman.pkl', 'rb') as f:
    fridman = pickle.load(f)

with open('Genesets/sasp_factors.pkl', 'rb') as f:
    sasp_factors = pickle.load(f)

mt_genes = [x for x in common_genes if x.startswith("MT-")]

In [7]:
sasp_factors = [x for x in sasp_factors if x in common_genes]
fridman = [x for x in fridman if x in common_genes]
senmayo = [x for x in senmayo if x in common_genes]

In [8]:
sasp_means = []
fridman_means = []
senmayo_means = []
mt_means = []

for i, _ in enumerate(st_dfs):
    sasp_means.append(st_dfs[i][sasp_factors].mean(axis = 1))
    fridman_means.append(st_dfs[i][fridman].mean(axis = 1))
    senmayo_means.append(st_dfs[i][senmayo].mean(axis = 1))
    mt_means.append(st_dfs[i][mt_genes].mean(axis = 1))

## Grid Search Setup

In [9]:
#max_features_range = (20, 5000)
max_features_range = (15, 3000)
max_depth_range = (1, 15)
max_samples_range = (0.05, 1.0)
threshold_range = (0.5, 0.7)

vars = ['max_features', 'max_depth', 'max_samples', 'threshold']

single_df_metrics = [
           'frid_t', 'frid_p',
           'senm_t', 'senm_p',
           'sasp_t', 'sasp_p',
           'mt_t', 'mt_p',
           'pct_tot_iso', 'pct_pos_iso', 'percent_pos', 'score', 'moran', 'geary']

metrics = []
for i, _ in enumerate(st_dfs):
    metrics = metrics + [x + f'_{i}' for x in single_df_metrics]

In [10]:
#grid_search_df = pd.DataFrame(
#    columns = vars + metrics
#)
with open('randomgridsearch.pkl', 'rb') as f:
    grid_search_df = pickle.load(f)

In [11]:
grid_search_df

,max_features,max_depth,max_samples,threshold,frid_t_0,frid_p_0,senm_t_0,senm_p_0,sasp_t_0,sasp_p_0,...,sasp_t_1,sasp_p_1,mt_t_1,mt_p_1,pct_tot_iso_1,pct_pos_iso_1,percent_pos_1,score_1,moran_1,geary_1
0,999.0,1.0,0.867002,0.614782,8.569816,1.336470e-11,8.315957,3.547737e-11,9.949983,1.462481e-13,...,4.460405,2.071616e-05,-4.284447,0.999965,0.014401,0.603774,0.023852,1.0000,0.331514,0.670524
1,1192.0,11.0,0.752816,0.671348,13.989771,2.838551e-35,14.873318,5.077183e-38,16.675979,1.200693e-45,...,8.686738,6.639783e-17,-6.726417,1.000000,0.036904,0.288732,0.127813,1.0000,0.303547,0.696227
2,822.0,15.0,0.720585,0.620369,15.294982,2.714264e-44,17.371458,1.620094e-53,20.415163,3.548231e-69,...,9.443321,6.172679e-20,-7.039744,1.000000,0.039604,0.234667,0.168767,1.0000,0.340183,0.660708
3,271.0,3.0,0.716765,0.570216,15.877813,2.106957e-46,17.279329,5.900873e-52,19.588677,2.161746e-63,...,11.977510,3.004452e-29,-4.839693,0.999999,0.034203,0.222222,0.153915,1.0000,0.408553,0.593404
4,161.0,6.0,0.359537,0.581143,10.749819,2.023286e-25,13.728748,2.434738e-38,17.099897,1.647365e-55,...,10.748724,1.373474e-24,-5.247040,1.000000,0.040054,0.244505,0.163816,0.9875,0.384098,0.618869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5798,307.0,11.0,0.213891,0.597800,15.203346,7.687970e-44,15.968315,1.118215e-46,19.162284,3.301890e-62,...,11.753849,1.598578e-27,-4.441780,0.999994,0.033753,0.265957,0.126913,1.0000,0.361730,0.640384
5799,63.0,6.0,0.785234,0.559132,14.660155,1.736359e-45,9.507695,6.009828e-21,9.188334,9.271745e-20,...,19.033313,1.926503e-67,1.862513,0.031424,0.030153,0.130350,0.231323,0.9875,0.433206,0.569904
5800,360.0,13.0,0.104828,0.612668,10.415272,1.455032e-21,12.153724,3.300993e-27,14.465173,3.347365e-35,...,7.146644,1.003534e-11,-5.482960,1.000000,0.031053,0.423313,0.073357,1.0000,0.376446,0.625292
5801,29.0,7.0,0.478414,0.650921,8.243693,1.686746e-08,6.445659,9.685089e-07,8.222468,1.999274e-08,...,6.063324,2.815130e-07,-0.587002,0.719596,0.011701,0.722222,0.016202,0.9500,0.495049,0.508283


## Grid Search

In [12]:
# Function Definitions
def get_neighbours(barcode, obs):
    x = obs.loc[barcode].array_row
    y = obs.loc[barcode].array_col
    neighbours = [
        (x, y-2),
        (x, y+2),
        (x-1, y-1),
        (x-1, y+1),
        (x+1, y-1),
        (x+1, y+1)
    ]
    mask = np.zeros(obs.shape[0]).astype(bool)

    for n in neighbours:
        mask = np.logical_or(
            mask, 
            np.logical_and(
                obs['array_row'] == n[0],
                obs['array_col'] == n[1]
            )
        )
    return mask[mask].index

def get_isolated(positives, obs):
    num_isolated = 0

    for pos in positives:
        neighbours = get_neighbours(pos, obs)
        if len(positives.intersection(neighbours)) == 0:
            num_isolated += 1

    return num_isolated

def set_cells(probs, threshold, means, geneset_name, idx, st_i):
    # Perfom t_test
    ttest = ttest_ind(
        means[probs[probs > threshold].index],
        means[probs[probs <= threshold].index],
        alternative = 'greater',
        equal_var = False
    )
    to_append.at[idx, f'{geneset_name}_t_{st_i}'] = ttest.statistic              # t-statistic
    to_append.at[idx, f'{geneset_name}_p_{st_i}'] = ttest.pvalue                 # p-value for t-statistic

In [13]:
warnings.filterwarnings('ignore')

ITERS_PER_SAVE = 2
gen_rv = lambda : int((expon.rvs() * (max_features_range[1]/7)) % (max_features_range[1] - max_features_range[0])) + max_features_range[0]
iterations = 0

while True:
    to_append = pd.DataFrame(
        data = np.zeros((ITERS_PER_SAVE, grid_search_df.shape[1])),
        columns = grid_search_df.columns
    )
                        
    for row_idx in range(0, ITERS_PER_SAVE):
        
        max_features = gen_rv()
        #max_features = rand.uniform(max_features_range[0], max_features_range[1])
        max_depth = rand.randint(max_depth_range[0], max_depth_range[1])
        max_samples = rand.uniform(max_samples_range[0], max_samples_range[1])
        threshold = rand.uniform(threshold_range[0], threshold_range[1])

        to_append.at[row_idx, 'max_features'] = max_features
        to_append.at[row_idx, 'max_depth'] = max_depth
        to_append.at[row_idx, 'max_samples'] = max_samples
        to_append.at[row_idx, 'threshold'] = threshold
        
        
        for i, _ in enumerate(st_dfs):
            df = scld_st_dfs[i]
            
            # Build the classifier for each ST dataset
            clf = RandomForestClassifier(
                n_estimators = 500,
                max_features = max_features,
                max_depth = max_depth,
                min_samples_split = 2,
                min_samples_leaf = 1,
                criterion = 'gini',
                ccp_alpha = 0.0,
                max_samples = max_samples,
                random_state = 42,
                n_jobs = 8
            )
    
            # Depending on if scaled or not, fit the scRNA-seq data pertinent to the given df and store that df as a var
            clf.fit(scld_trn_X[i], trn_y[i])
            score = clf.score(scld_tst_X[i], tst_y[i])

            # Store the score
            to_append.at[row_idx, f'score_{i}'] = score

            # Classify & store data
            probs = pd.Series([x[1] for x in clf.predict_proba(df[clf.feature_names_in_])], index = df.index)
            positives = probs[probs > threshold].index
            if (len(positives) == 0) or (len(positives) == len(probs)):
                to_append.loc[row_idx] = np.nan
                continue

            to_append.at[row_idx, f'percent_pos_{i}'] = len(positives) / len(probs)

            # Perform t-tests for each gene set of interest
            set_cells(probs, threshold, senmayo_means[i], 'senm', row_idx, i)
            if to_append.at[row_idx, f'senm_p_{i}'] > 0.1:
                to_append.loc[row_idx] = np.nan
                continue
                
            set_cells(probs, threshold, fridman_means[i], 'frid', row_idx, i)
            set_cells(probs, threshold, sasp_means[i], 'sasp', row_idx, i)
            set_cells(probs, threshold, mt_means[i], 'mt', row_idx, i)

            # Calculate the isolated spots
            isolated_spots = get_isolated(positives, st_obs[i])
            to_append.at[row_idx, f'pct_pos_iso_{i}'] = isolated_spots / len(positives)
            to_append.at[row_idx, f'pct_tot_iso_{i}'] = isolated_spots / len(probs)

            adatas[i].obs['probs'] = probs
            moran = sq.gr.spatial_autocorr(
                adatas[i],
                mode = 'moran',
                genes = 'probs',
                n_perms = 100,
                attr = 'obs',
                copy = True,
                show_progress_bar = False,
                n_jobs = 8
            )
            geary = sq.gr.spatial_autocorr(
                adatas[i],
                mode = 'geary',
                genes = 'probs',
                n_perms = 100,
                attr = 'obs',
                copy = True,
                show_progress_bar = False,
                n_jobs = 8
            )
            to_append.at[row_idx, f'moran_{i}'] = moran['I']
            to_append.at[row_idx, f'geary_{i}'] = geary['C']
    
    to_append.dropna(inplace = True, ignore_index = True)
    grid_search_df = pd.concat([grid_search_df, to_append], ignore_index = True, copy = False)

    with open('randomgridsearch.pkl', 'wb') as f:
        pickle.dump(grid_search_df, f)

    if grid_search_df.shape[0] > 100000:
        break

warnings.filterwarnings('default')

KeyboardInterrupt: 